# Introduction

The goal is to build the network of cross-holding to analyze financial contagion. The graph will be serialized in a pickle object.

The main dataset **df_shareholder** details the shareholder's ID for each company and the amout of capital held by each shareholder. Then, the RUC of all firms are collected using **df_directory**. Finally, we drop the firms that do not have at least another firm as a shareholder. The final dataset of cross-holdings is **df_firms**.

In [1]:
NAME = 'Prepare_Data'
PROJECT = 'Financial_Contagion_Crossholding'

### Imports  

In [2]:
# Load libraries
import pandas as pd
import numpy as np
import networkx as nx

# Ignoring warnings
import warnings
warnings.filterwarnings('ignore')

--- 
# Main code

In [3]:
data_path = "../0_data/"
read_path = "../2_pipeline/0_transform_kardex/"
write_path = "../2_pipeline/1_prepare_data/"
year = 2016

### Reading shareholder register

In [4]:
# Reading data
df_shareholder = pd.read_csv(read_path + "accionistas{}.csv".format(year),
                             encoding="latin1", 
                             dtype={"EXPEDIENTE": np.int32} )
df_shareholder.columns = ["Expediente", "Capital", "Accionista"]

# Dropping missing values
print("Missing values dropped:", len(df_shareholder[df_shareholder["Accionista"] == "9999999999998"]))
df_shareholder = df_shareholder[df_shareholder["Accionista"] != "9999999999998"]

Missing values dropped: 53


In [5]:
# Summary statistics for the total amount of shareholders in each company
df_shareholder.Expediente.value_counts().describe()

count    49452.000000
mean         6.200376
std         74.800678
min          1.000000
25%          2.000000
50%          2.000000
75%          3.000000
max      14757.000000
Name: Expediente, dtype: float64

## Adding RUC column

In [6]:
rama_dict = {
    "A": "Agriculture",
    "B": "General Services",
    "C": "Manufacturing",
    "D": "General Services",
    "E": "General Services",
    "F": "Construction",
    "G": "Trade",
    "H": "Transportation",
    "I": "General Services",
    "J": "General Services",
    "K": "Financial Activities",
    "L": "Real Estate",
    "M": "General Services",
    "N": "General Services",
    "P": "General Services",
    "Q": "General Services",
    "R": "General Services", 
    "S": "General Services"
}

In [7]:
# Reading firms' directory
df_directory = pd.read_table(data_path + "directorio_companias.txt", 
                             delimiter="\t", 
                             encoding="latin1")

# Missing values just in RUC, there is no problem to drop everything
print("Missing values dropped:\n", df_directory.isna().sum(), sep="")
df_directory.dropna(inplace=True)

# Modifying columns to look better
df_directory["CIIU"] = df_directory["Actividad_economica"].apply(lambda x: list(x)[0])
df_directory["Fecha_constitucion"] = pd.to_datetime(df_directory["Fecha_constitucion"])
df_directory["Fecha_constitucion"] = 2018 - df_directory["Fecha_constitucion"].dt.year
df_directory['Sector_Econ'] = df_directory['CIIU'].map(rama_dict)

Missing values dropped:
Expediente                0
Ruc                    2565
Nombre                    0
Fecha_constitucion        0
Provincia                 0
Canton                    0
Actividad_economica       0
dtype: int64


In [8]:
# Problem: RUC number is a float, so first I need to convert it in integer, and then in string
df_directory = df_directory.astype({'Ruc': 'int64'}).astype({'Ruc': 'str'})

# Adding Ruc column to df_shareholder
df_shareholder = df_shareholder.merge(df_directory[["Expediente", "Ruc"]], how='inner', on="Expediente")

## Validating financial info

In [9]:
def read_fin_info(loc_path, column):
    """
    This function read financial datasets and drop firms with zero assets
    """
  
    # Reading financial information for firms
    df_finance = pd.read_stata(loc_path)
    # Adjusting name of Ruc
    df_finance = df_finance.rename({"RUC": "Ruc"}, axis=1)
    # Problem: RUC number is a float, so first I need to convert it in integer, and then in string
    df_finance = df_finance.astype({'Ruc': 'int64'}).astype({'Ruc': 'str'})
    # Drop firms with zero assets
    print("Firms with zero assets:", len(df_finance[ df_finance["TOTAL_ACTIVO"] <= 0 ]) )
    df_finance = df_finance[ df_finance["TOTAL_ACTIVO"] > 0 ]
    
    return df_finance

In [10]:
# Reading financial information for firms
df_finance = read_fin_info(data_path + "balance{}.dta".format(year), 
                           column="TOTAL_ACTIVO")

# Filtering data by financial information of 2016
finance_set = set(df_finance.Ruc)#.intersection( set(df_finance_last.Ruc) )
df_shareholder = df_shareholder[df_shareholder.Ruc.isin( finance_set )]

Firms with zero assets: 1807


In [11]:
df_finance.columns

Index(['AÑO', 'EXPEDIENTE', 'Ruc', 'NOMBRE', 'CIIU', 'EFECTIVO',
       'INVERSIONES_FINANCIERAS', 'CTAS_Y_DOC_X_COBRAR', 'INVENTARIOS',
       'TOTAL_ACTIVOS_CORRIENTES', 'ACTIVO_FIJO_TANGIBLE',
       'ACTIVO_FIJO_INTANGIBLE', 'TOTAL_ACTIVO_FIJO', 'TOTAL_ACTIVO',
       'CTAS_Y_DOC_X_PAGAR', 'OBLIGACIONES_FINANCIERAS',
       'TOTAL_PASIVO_CORRIENTE', 'TOTAL_PASIVO', 'CAPITAL',
       'UTILIDADES_ANTERIORES', 'TOTAL_PATRIMONIO', 'VENTAS', 'EXPORTACIONES',
       'TOTAL_INGRESOS', 'TOTAL_COSTO_DE_VENTA', 'SUELDOS', 'ARRENDAMIENTO',
       'PUBLICIDAD', 'SERVICIOS_BASICOS', 'IMPUESTOS_Y_CONTRIBUCIONES',
       'INTERESES', 'UTILIDAD_DEL_EJERCICIO', 'TOTAL_EGRESOS',
       'IMPUESTO_A_LA_RENTA'],
      dtype='object')

In [12]:
# Create Economic Sector
df_finance["Actividad_economica"] = df_finance["CIIU"].apply(lambda x: list(x)[0])
df_finance["Sector_Econ"] = df_finance["Actividad_economica"].map(rama_dict)
# Filter finance data by the RUCs in shareholder register
df_finance = df_finance[df_finance["Ruc"].isin(set(df_shareholder["Ruc"]))]

In [13]:
# df_finance[["TOTAL_ACTIVO", "Sector_Econ"]].groupby("Sector_Econ").sum().to_excel("total_activos_industria.xlsx")

## Filtering cross-holdings

In [14]:
def stats_comunes(data, column_shareholder, column_firm):
    share_total_set = set(data[column_shareholder])
    firms_total_set = set(data[column_firm])

    print("Unique shareholders: ", len(share_total_set))
    print("Unique firms: ", len(firms_total_set))
    print("Firms that are not shareholders: ", len( firms_total_set.difference(share_total_set) ))
    print("Shareholders that are not firms: ", len( share_total_set.difference(firms_total_set) ))
    print("Intersection between firms and shareholders: ", len( firms_total_set.intersection(share_total_set) ))

In [15]:
stats_comunes(df_shareholder, "Accionista", "Ruc")

Unique shareholders:  217407
Unique firms:  44104
Firms that are not shareholders:  42805
Shareholders that are not firms:  216108
Intersection between firms and shareholders:  1299


In [16]:
firms_holders_set = set(df_shareholder["Accionista"]).intersection(set(df_shareholder["Ruc"]))
print("Firms for analysis:", len(firms_holders_set))

Firms for analysis: 1299


In [17]:
# Creating a dataframe with cross-holdings in firms
df_firms = df_shareholder[ df_shareholder.Ruc.isin(firms_holders_set) ]

# Calculating percentage of share capital
expediente_accionista = df_firms.groupby(['Expediente', 'Accionista']).agg({'Capital': 'sum'})
expediente_capital = expediente_accionista.groupby(level=0).sum().reset_index().rename({"Capital": "Total_Equity"}, axis=1)
df_firms = expediente_accionista.groupby(level=0).apply(lambda x: x / float(x.sum()))
df_firms = df_firms.reset_index()

# Adding again RUC column to new df_firms
df_firms = df_firms.merge(df_directory[["Expediente", "Ruc"]], how='inner', on="Expediente")
df_firms = df_firms.merge(expediente_capital, how='inner', on="Expediente")

In [18]:
# Reading financial information for firms
# df_ceo = pd.read_stata(os.path.join('..', '0_data', "BDD_GENRED.dta"))
# df_ceo

## Building network

In [19]:
weight_list = list( zip( df_firms.Accionista.astype('str'), df_firms.Ruc.astype('str'), df_firms.Capital ) )

G = nx.DiGraph( name='Cross-holding Network' )
G.add_weighted_edges_from( weight_list )

print('Graph information\n', nx.info(G), sep='')

Graph information
Name: Cross-holding Network
Type: DiGraph
Number of nodes: 37012
Number of edges: 41578
Average in degree:   1.1234
Average out degree:   1.1234


In [20]:
# Remove nodes that are not cross-holding firms
G.remove_nodes_from([n for n in G if n not in firms_holders_set])

print('Graph information\n', nx.info(G), sep='')

Graph information
Name: Cross-holding Network
Type: DiGraph
Number of nodes: 1299
Number of edges: 777
Average in degree:   0.5982
Average out degree:   0.5982


In [21]:
# Remove self-loops
print("Remove {} self-loops \n".format(nx.number_of_selfloops(G)))

G.remove_edges_from(nx.selfloop_edges(G))
print('Graph information\n', nx.info(G), sep='')

Remove 156 self-loops 

Graph information
Name: Cross-holding Network
Type: DiGraph
Number of nodes: 1299
Number of edges: 621
Average in degree:   0.4781
Average out degree:   0.4781


In [22]:
# Creating dictionary of attributes
ruc_econSector = df_directory[df_directory.Ruc.isin(G.nodes)].set_index("Ruc").Sector_Econ.to_dict()
ruc_years = df_directory[df_directory.Ruc.isin(G.nodes)].set_index("Ruc").Fecha_constitucion.to_dict()
ruc_name = df_directory[df_directory.Ruc.isin(G.nodes)].set_index("Ruc").Nombre.to_dict()

ruc_assets = df_finance[df_finance.Ruc.isin(G.nodes)].set_index("Ruc").TOTAL_ACTIVO.to_dict()
ruc_equity = df_firms[df_firms.Ruc.isin(G.nodes)].set_index("Ruc").Total_Equity.to_dict()
ruc_revenue = df_finance[df_finance.Ruc.isin(G.nodes)].set_index("Ruc").TOTAL_INGRESOS.to_dict()

df_total_shareholders = df_shareholder.Ruc.value_counts().reset_index()
df_total_shareholders.columns = ["Ruc", "Total_shareholders"]
ruc_shareholders = df_total_shareholders[df_total_shareholders.Ruc.isin(G.nodes)].set_index("Ruc").Total_shareholders.to_dict()

# Adding attributes to firms
nx.set_node_attributes(G, ruc_econSector, name="Economic_Sector")
nx.set_node_attributes(G, ruc_years, name="Years")
nx.set_node_attributes(G, ruc_name, name="Name")
nx.set_node_attributes(G, ruc_assets, name="Total_Assets")
nx.set_node_attributes(G, ruc_equity, name="Total_Equity")
nx.set_node_attributes(G, ruc_revenue, name="Total_Revenue")
nx.set_node_attributes(G, ruc_shareholders, name="Total_Shareholders")

In [23]:
nx.write_gpickle(G, write_path + "graph_crossholding_{}.gpickle".format(year))